<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 3. Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>5. Proyecto de clasificación multiclase</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Importar clases, funciones y conjunto de datos](#section1)
* [2. Codificar la variable de salida](#section2)
* [3. Definir la red neuronal](#section3)
* [4. Evaluar el modelo](#section4)
* [5. Código completo](#section5)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En este tutorial del proyecto, descubrirá cómo puede usar Keras para desarrollar y evaluar modelos de redes neuronales para problemas de clasificación multiclase. Después de completar este tutorial paso a paso, sabrá:
* Cómo cargar datos desde CSV y ponerlos a disposición de Keras.
* Cómo preparar datos de clasificación multiclase para modelar con redes neuronales.
* Cómo evaluar modelos de redes neuronales de Keras con scikit-learn.

In [7]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Importar clases, funciones y conjunto de datos</font>

Podemos comenzar importando todas las clases, funciones y dataset que necesitaremos en este tutorial. Esto incluye tanto la funcionalidad que requerimos de Keras como la carga de datos de Pandas, así como la preparación de datos y la evaluación de modelos de scikit-learn.

### Modificaciones
He cambiado la importación del wrapper y del to_categorical

In [1]:
# Multiclass Classification with the Iris Flowers Dataset
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
## La nueva versión keras es con scikeras
from scikeras.wrappers import KerasClassifier, KerasRegressor
#from keras.utils import np_utils
# LA nueva versión keras has sustituido to_categorical así:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
# load dataset
dataframe = pd.read_csv("Datasets/iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

/Users/manwest/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el dataset [Iris](https://archive.ics.uci.edu/ml/datasets/Iris)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Codificar la variable de salida</font>

La variable de salida contiene tres valores de cadena diferentes. Al modelar problemas de clasificación multiclase utilizando redes neuronales, es una buena práctica remodelar el atributo de salida de un vector que contiene valores para cada valor de clase para que sea una matriz con un booleano para cada valor de clase y si una instancia determinada tiene o no ese valor de clase. o no. Esto se denomina One-Hot Encoding. 

Por ejemplo, en este problema los tres valores de clase son Iris-setosa, Iris-versicolor e Iris-virginica. Si tuviéramos las tres observaciones:
```
    Iris-setosa
    Iris-versicolor
    Iris-virginica
```

Podemos convertir esto en una matriz binaria codificada en caliente para cada instancia de datos que se vería de la siguiente manera:
```
    Iris-setosa, Iris-versicolor, Iris-virginica
        1,             0,              0
        0,             1,              0
        0,             0,              1
```
Podemos hacer esto codificando primero las cadenas de manera coherente en números enteros utilizando la clase `LabelEncoder` de scikit-learn. Luego, convierta el vector de números enteros en One-Hot Enconding usando la función de Keras `to_categorical()`.

### Aquí he cambiado to_categorical

In [2]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
print(encoded_Y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [3]:

# convert integers to dummy variables (i.e. one hot encoded)
#dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y = to_categorical(encoded_Y)
print(dummy_y)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Definir la red neuronal</font>

La biblioteca de Keras proporciona clases contenedoras para permitirle usar modelos de redes neuronales desarrollados con Keras en scikit-learn como vimos en la lección anterior. Hay una clase `KerasClassifier` en Keras que se puede usar como Estimador en scikit-learn. `KerasClassifier` toma el nombre de una función como argumento. Esta función debe devolver el modelo de red neuronal construido, listo para el entrenamiento.

A continuación se muestra una función que creará una red neuronal de referencia para el problema de clasificación de Iris. 
1. Crea una red simple completamente conectada con una capa oculta que contiene 8 neuronas. 
2. La capa oculta utiliza una función de activación ReLu, lo cual es una buena práctica. 
3. Debido a que utilizamos una One-Hot Encoding para nuestro conjunto de datos de Iris, la capa de salida debe crear 3 valores de salida, uno para cada clase. 
4. El valor de salida con el valor más grande se tomará como la clase predicha por el modelo. La topología de la red de esta simple red neuronal de una capa se puede resumir como:
```
    4 inputs -> [8 hidden nodes] -> 3 outputs
```
5. Tenga en cuenta que usamos una función de activación Softmax en la capa de salida. Esto es para asegurar que los valores de salida estén en el rango de 0 y 1 y puedan usarse como probabilidades predichas. 
6. Finalmente, la red utiliza el eficiente algoritmo de optimización Gradiente Descendiente de Adam con una función de pérdida logarítmica, que se denomina `categorical_crossentropy` en Keras.

In [4]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Ahora podemos crear nuestro `KerasClassifier` para usarlo en scikit-learn. También podemos pasar argumentos en la construcción de la clase `KerasClassifier` que se pasarán a la función `fit()` utilizada internamente para entrenar la red neuronal. Aquí, pasamos el número de épocas como 200 y el tamaño del lote como 5 para usar al entrenar el modelo. 

In [5]:
## Ha cambiado build_fn por el parámetro model
estimator = KerasClassifier(model=baseline_model, epochs=50, batch_size=5, verbose=2)

<a id="section4"></a>
# <font color="#004D7F" size=6>4. Evaluar el modelo</font>

Ahora podemos evaluar nuestro modelo (estimador) en nuestro conjunto de datos (`X` e `dummy_y`) utilizando un procedimiento de validación cruzada de 10 veces (`kfold`). Evaluar el modelo solo toma aproximadamente 10 segundos y devuelve un objeto que describe la evaluación de los 10 modelos construidos para cada una de las divisiones del conjunto de datos.

La lista completa de códigos se proporciona a continuación para completar. Los resultados se resumen como la desviación media y estándar del Accuracy del modelo en el conjunto de datos. Ésta es una estimación razonable del rendimiento del modelo en datos no etiquetados. 

In [6]:
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/50


/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-09-26 17:31:41.876204: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-09-26 17:31:41.876225: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-09-26 17:31:41.876230: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-09-26 17:31:41.876247: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-26 17:31:41.876260: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271]

24/24 - 3s - 141ms/step - accuracy: 0.3333 - loss: 1.2006
Epoch 2/50
24/24 - 0s - 7ms/step - accuracy: 0.3333 - loss: 1.0863
Epoch 3/50
24/24 - 0s - 7ms/step - accuracy: 0.3500 - loss: 1.0104
Epoch 4/50
24/24 - 0s - 7ms/step - accuracy: 0.4417 - loss: 0.9559
Epoch 5/50
24/24 - 0s - 7ms/step - accuracy: 0.4500 - loss: 0.9095
Epoch 6/50
24/24 - 0s - 7ms/step - accuracy: 0.7667 - loss: 0.8424
Epoch 7/50
24/24 - 0s - 7ms/step - accuracy: 0.8000 - loss: 0.8061
Epoch 8/50
24/24 - 0s - 7ms/step - accuracy: 0.8417 - loss: 0.7787
Epoch 9/50
24/24 - 0s - 7ms/step - accuracy: 0.8750 - loss: 0.7522
Epoch 10/50
24/24 - 0s - 7ms/step - accuracy: 0.8750 - loss: 0.7270
Epoch 11/50
24/24 - 0s - 8ms/step - accuracy: 0.8417 - loss: 0.7052
Epoch 12/50
24/24 - 0s - 8ms/step - accuracy: 0.8667 - loss: 0.6784
Epoch 13/50
24/24 - 0s - 7ms/step - accuracy: 0.8500 - loss: 0.6569
Epoch 14/50
24/24 - 0s - 7ms/step - accuracy: 0.8750 - loss: 0.6352
Epoch 15/50
24/24 - 0s - 7ms/step - accuracy: 0.8417 - loss: 0.612

/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 - 1s - 27ms/step - accuracy: 0.3417 - loss: 2.5909
Epoch 2/50
24/24 - 0s - 8ms/step - accuracy: 0.3417 - loss: 2.0125
Epoch 3/50
24/24 - 0s - 8ms/step - accuracy: 0.3417 - loss: 1.6279
Epoch 4/50
24/24 - 0s - 8ms/step - accuracy: 0.3417 - loss: 1.3686
Epoch 5/50
24/24 - 0s - 8ms/step - accuracy: 0.3417 - loss: 1.2084
Epoch 6/50
24/24 - 0s - 9ms/step - accuracy: 0.3417 - loss: 1.0944
Epoch 7/50
24/24 - 0s - 8ms/step - accuracy: 0.3417 - loss: 1.0140
Epoch 8/50
24/24 - 0s - 8ms/step - accuracy: 0.3417 - loss: 0.9513
Epoch 9/50
24/24 - 0s - 8ms/step - accuracy: 0.3417 - loss: 0.9008
Epoch 10/50
24/24 - 0s - 8ms/step - accuracy: 0.3667 - loss: 0.8602
Epoch 11/50
24/24 - 0s - 8ms/step - accuracy: 0.4000 - loss: 0.8306
Epoch 12/50
24/24 - 0s - 8ms/step - accuracy: 0.4000 - loss: 0.8057
Epoch 13/50
24/24 - 0s - 8ms/step - accuracy: 0.4333 - loss: 0.7846
Epoch 14/50
24/24 - 0s - 8ms/step - accuracy: 0.4750 - loss: 0.7669
Epoch 15/50
24/24 - 0s - 8ms/step - accuracy: 0.4750 - loss: 0.7506

...

24/24 - 1s - 33ms/step - accuracy: 0.3500 - loss: 1.3600
Epoch 2/50
24/24 - 0s - 7ms/step - accuracy: 0.1000 - loss: 1.2792
Epoch 3/50
24/24 - 0s - 7ms/step - accuracy: 0.0083 - loss: 1.2042
Epoch 4/50
24/24 - 0s - 8ms/step - accuracy: 0.0000e+00 - loss: 1.1434
Epoch 5/50
24/24 - 0s - 7ms/step - accuracy: 0.0333 - loss: 1.1018
Epoch 6/50
24/24 - 0s - 7ms/step - accuracy: 0.2583 - loss: 1.0818
Epoch 7/50
24/24 - 0s - 7ms/step - accuracy: 0.3000 - loss: 1.0791
Epoch 8/50
24/24 - 0s - 7ms/step - accuracy: 0.2750 - loss: 1.0783
Epoch 9/50
24/24 - 0s - 7ms/step - accuracy: 0.2750 - loss: 1.0772
Epoch 10/50
24/24 - 0s - 7ms/step - accuracy: 0.2833 - loss: 1.0762
Epoch 11/50
24/24 - 0s - 7ms/step - accuracy: 0.2750 - loss: 1.0753
Epoch 12/50
24/24 - 0s - 7ms/step - accuracy: 0.2750 - loss: 1.0741
Epoch 13/50
24/24 - 0s - 8ms/step - accuracy: 0.2750 - loss: 1.0732
Epoch 14/50
24/24 - 0s - 8ms/step - accuracy: 0.2667 - loss: 1.0714
Epoch 15/50
24/24 - 0s - 7ms/step - accuracy: 0.2250 - loss: 1.

/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 - 1s - 26ms/step - accuracy: 0.3333 - loss: 5.9671
Epoch 2/50
24/24 - 0s - 7ms/step - accuracy: 0.3333 - loss: 4.6574
Epoch 3/50
24/24 - 0s - 7ms/step - accuracy: 0.3333 - loss: 3.6158
Epoch 4/50
24/24 - 0s - 8ms/step - accuracy: 0.3333 - loss: 2.7591
Epoch 5/50
24/24 - 0s - 7ms/step - accuracy: 0.3333 - loss: 2.1272
Epoch 6/50
24/24 - 0s - 7ms/step - accuracy: 0.3333 - loss: 1.8044
Epoch 7/50
24/24 - 0s - 7ms/step - accuracy: 0.3333 - loss: 1.5628
Epoch 8/50
24/24 - 0s - 9ms/step - accuracy: 0.3333 - loss: 1.4058
Epoch 9/50
24/24 - 0s - 7ms/step - accuracy: 0.3333 - loss: 1.2985
Epoch 10/50
24/24 - 0s - 7ms/step - accuracy: 0.3333 - loss: 1.2258
Epoch 11/50
24/24 - 0s - 8ms/step - accuracy: 0.3333 - loss: 1.1784
Epoch 12/50
24/24 - 0s - 7ms/step - accuracy: 0.3333 - loss: 1.1476
Epoch 13/50
24/24 - 0s - 7ms/step - accuracy: 0.3333 - loss: 1.1265
Epoch 14/50
24/24 - 0s - 7ms/step - accuracy: 0.3333 - loss: 1.1105
Epoch 15/50
24/24 - 0s - 7ms/step - accuracy: 0.3333 - loss: 1.0953

/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 - 1s - 25ms/step - accuracy: 0.1583 - loss: 3.1394
Epoch 2/50
24/24 - 0s - 7ms/step - accuracy: 0.0083 - loss: 2.4961
Epoch 3/50
24/24 - 0s - 7ms/step - accuracy: 0.0083 - loss: 1.9638
Epoch 4/50
24/24 - 0s - 7ms/step - accuracy: 0.1583 - loss: 1.6095
Epoch 5/50
24/24 - 0s - 7ms/step - accuracy: 0.1417 - loss: 1.4101
Epoch 6/50
24/24 - 0s - 7ms/step - accuracy: 0.1167 - loss: 1.2632
Epoch 7/50
24/24 - 0s - 7ms/step - accuracy: 0.2417 - loss: 1.1964
Epoch 8/50
24/24 - 0s - 7ms/step - accuracy: 0.3167 - loss: 1.1128
Epoch 9/50
24/24 - 0s - 7ms/step - accuracy: 0.3500 - loss: 1.0373
Epoch 10/50
24/24 - 0s - 7ms/step - accuracy: 0.6083 - loss: 0.9761
Epoch 11/50
24/24 - 0s - 7ms/step - accuracy: 0.6917 - loss: 0.9227
Epoch 12/50
24/24 - 0s - 7ms/step - accuracy: 0.6333 - loss: 0.8892
Epoch 13/50
24/24 - 0s - 7ms/step - accuracy: 0.5917 - loss: 0.8646
Epoch 14/50
24/24 - 0s - 7ms/step - accuracy: 0.6583 - loss: 0.8400
Epoch 15/50
24/24 - 0s - 7ms/step - accuracy: 0.6500 - loss: 0.8182

<a id="section5"></a>
# <font color="#004D7F" size=6>5. Código completo</font>

In [1]:
# Multiclass Classification with the Iris Flowers Dataset
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
# load dataset
dataframe = pd.read_csv("Datasets/iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

2022-09-07 09:55:28.500146: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-07 09:55:28.500245: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/tmp/ipykernel_5567/1943733779.py:30: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
2022-09-07 09:55:48.119377: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-07 09:55:48.119483: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be runn

Accuracy: 96.67% (4.47%)


<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>